In [105]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test', 'train', 'train.csv', 'sample_submission.csv']


In [106]:
import os
import sys
import numpy as np
import pandas as pd
import zipfile
import keras
from PIL import Image
from keras_applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [107]:
print(os.listdir("../input/train/train")[0])
path = os.path.join('../input/train/train', os.listdir("../input/train/train")[0])
print(path)
print(load_img(path))

028192187883168e2a7621c998dc447a.jpg
../input/train/train/028192187883168e2a7621c998dc447a.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=32x32 at 0x7FF3AE595550>


In [108]:
class DataHandler():
    def __init__(self, csv_name):
        self.dataset_path = '../input'
        print(os.listdir(self.dataset_path))
        self.train_data = pd.read_csv(os.path.join(self.dataset_path, csv_name))
        # zipfiles = ['test.zip', 'train.zip']
        # for datas in zipfiles:
        #     target_dir = os.path.join(self.dataset_path, datas.split('.')[0])
        #     os.makedirs(target_dir, exist_ok=True)
        #     with zipfile.ZipFile(os.path.join(self.dataset_path, datas)) as f:
        #         f.extractall(target_dir)

    def get_from_columns(self, *args):
        for column in args:
            yield self.train_data[column]

    def get_only_data(self, folname):
        fol_path = os.path.join(self.dataset_path, folname)
        img_paths = list(map(lambda fname: os.path.join(fname), os.listdir(fol_path)))
        return img_paths

    def get_data_label(self, folname='train/train', val_rate=0.2):
        fol_path = os.path.join(self.dataset_path, folname)
        params = self.train_data.columns.values
        datas = []
        labels = []
        if folname.split('/')[0] == 'train':
            for idx ,row in tqdm(self.train_data.iterrows()):
                for data_or_label, param in enumerate(params):
                    if data_or_label == 0:
                        img_name = row[param]
                        img_data = img_to_array(load_img(os.path.join(fol_path, img_name)))
                        datas.append(img_data)
                    else:
                        labels.append(float(row[param]))
        else:
            img_names = os.listdir(fol_path)
            self.test_names = img_names
            for img_name in tqdm(img_names):
                img_data = img_to_array(load_img(os.path.join(fol_path, img_name)))
                datas.append(img_data)
            datas = np.asarray(datas).astype(np.float)
            datas /= 255
            print(len(datas))
            return datas

        datas = np.asarray(datas).astype(np.float)
        datas /= 255
        labels = np.asarray(labels).astype(np.float)
        datas, val_datas, labels, val_labels = train_test_split(datas, labels, test_size=val_rate)
        return (datas, labels, val_datas, val_labels)

    def get_shape(self, folname='train/train'):
        from random import randint
        fol_path = os.path.join(self.dataset_path, folname)
        sample_img_path = os.path.join(fol_path ,os.listdir(fol_path)[randint(0,20)])
        img_bin = img_to_array(load_img(sample_img_path)) # return ndarray
        return img_bin.shape

In [109]:
class Train():
    def __init__(self, use_imgnt=False):
        pass

    def build_model(self, img_shape, do_cb=True):
        self.model = Sequential()
        self.model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=img_shape))
        self.model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2,2)))
        self.model.add(Dropout(0.1))
        self.model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
        self.model.add(Conv2D(filters=64, kernel_size=(5, 3), padding='same', activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.1))
        self.model.add(Flatten())
        self.model.add(Dense(1, activation='sigmoid'))
        self.model.compile(
            optimizer=keras.optimizers.Adam(),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        if do_cb == True:
            self.reduce_lr = ReduceLROnPlateau(monitor='val_loss', verbose=1)
            self.early_stopping = EarlyStopping(monitor='val_loss', verbose=1, mode='auto')
        return self.model.summary

    def train(self, x_train, y_train, x_test, y_test, epochs=20, batch_size=32):
        hist = self.model.fit(
            x=x_train, y=y_train,
            # steps_per_epoch=x_train.shape[0] // batch_size,
            # validation_steps=x_test.shape[0] // batch_size,
            epochs=epochs,
            batch_size=batch_size,
            verbose=1,
            validation_data=(x_test, y_test),
        )
        return hist
    
    def predict(self, test):
        pred = self.model.predict(
            test
        )
        return pred

In [110]:
data_handler = DataHandler('train.csv')
datagen = data_handler.get_from_columns('id', 'has_cactus')
img_shape = data_handler.get_shape()
test = data_handler.get_data_label('test/test')
test_names = data_handler.test_names
print(len(test), len(test_names))
x_train, y_train, x_test, y_test = data_handler.get_data_label('train/train')
model = Train()
summary = model.build_model(img_shape=img_shape)
summary()
hist = model.train(x_train, y_train, x_test, y_test)



  0%|          | 0/4000 [00:00<?, ?it/s]

['test', 'train', 'train.csv', 'sample_submission.csv']




  1%|          | 47/4000 [00:00<00:08, 468.29it/s]

  2%|▏         | 85/4000 [00:00<00:08, 435.27it/s]

  3%|▎         | 123/4000 [00:00<00:09, 414.15it/s]

  4%|▍         | 162/4000 [00:00<00:09, 404.63it/s]

  5%|▍         | 196/4000 [00:00<00:09, 380.63it/s]

  6%|▌         | 233/4000 [00:00<00:09, 376.95it/s]

  7%|▋         | 268/4000 [00:00<00:10, 363.78it/s]

  8%|▊         | 306/4000 [00:00<00:10, 368.01it/s]

  9%|▊         | 345/4000 [00:00<00:09, 368.69it/s]

 10%|▉         | 381/4000 [00:01<00:10, 359.43it/s]

 10%|█         | 418/4000 [00:01<00:09, 360.42it/s]

 11%|█▏        | 455/4000 [00:01<00:09, 361.46it/s]

 12%|█▏        | 497/4000 [00:01<00:09, 376.58it/s]

 13%|█▎        | 539/4000 [00:01<00:09, 383.68it/s]

 14%|█▍        | 578/4000 [00:01<00:08, 381.19it/s]

 15%|█▌        | 617/4000 [00:01<00:08, 379.57it/s]

 16%|█▋        | 655/4000 [00:01<00:09, 358.63it/s]

 17%|█▋        | 696/4000 [00:01<00:08, 371.47it/s]

 18%|█▊        | 734/4000 [00:01<00:09, 362.08

4000
4000 4000




189it [00:00, 930.40it/s]

283it [00:00, 931.86it/s]

381it [00:00, 945.72it/s]

480it [00:00, 958.04it/s]

570it [00:00, 939.55it/s]

660it [00:00, 924.73it/s]

746it [00:00, 876.25it/s]

840it [00:00, 892.41it/s]

929it [00:01, 891.25it/s]

1024it [00:01, 905.83it/s]

1121it [00:01, 918.22it/s]

1212it [00:01, 909.50it/s]

1308it [00:01, 923.13it/s]

1404it [00:01, 930.76it/s]

1500it [00:01, 938.16it/s]

1594it [00:01, 896.08it/s]

1684it [00:01, 802.75it/s]

1767it [00:01, 776.39it/s]

1854it [00:02, 801.04it/s]

1944it [00:02, 827.17it/s]

2038it [00:02, 856.66it/s]

2125it [00:02, 841.53it/s]

2221it [00:02, 872.70it/s]

2310it [00:02, 877.81it/s]

2399it [00:02, 871.28it/s]

2498it [00:02, 903.41it/s]

2589it [00:02, 883.40it/s]

2683it [00:03, 897.78it/s]

2774it [00:03, 873.26it/s]

2869it [00:03, 894.91it/s]

2960it [00:03, 895.78it/s]

3050it [00:03, 868.35it/s]

3142it [00:03, 880.92it/s]

3231it [00:03, 883.18it/s]

3324it [00:03, 894.98it/s]

3419it [00:03, 910.56it/s]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_53 (Conv2D)           (None, 32, 32, 32)        2432      
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 32, 32, 32)        25632     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 16, 16, 64)        61504     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 8, 8, 64)          0         
__________

In [111]:
predict = model.predict(test)

In [112]:
submission_df = pd.DataFrame(predict, columns=['has_cactus'])
submission_df['id'] = ''
cols = submission_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
submission_df = submission_df[cols]
for i, name in enumerate(test_names):
    submission_df.set_value(i, 'id', name)
submission_df.head(10)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


,id,has_cactus
0,c662bde123f0f83b3caae0ffda237a93.jpg,1.000000e+00
1,9553eed7793d4cf88b5226d446d93dae.jpg,8.940697e-08
2,19f059a7ce41b25be1548bc4049b45ec.jpg,1.000000e+00
3,fb4f464486f4894330273346ce939252.jpg,1.000000e+00
4,b52558a522db6ec2501ae188b6d6e526.jpg,8.447138e-01
5,d91989075ade0ecab01c9b5cae1c00d3.jpg,1.000000e+00
6,e6083ba00ecd6c4b0a50cd730dd9aa1f.jpg,9.989497e-01
7,3c2504a050b6e034c09a07d9846bce16.jpg,1.000000e+00
8,3e93a4783dcbd7b881d872a68d180515.jpg,3.100276e-03
9,4a1b4330fa36c223c13b3169ed8e912a.jpg,1.000000e+00


In [113]:
submission_df.to_csv('submission.csv', index=False)